| Column Name             | Data Type | Description                                              | null |
|-------------------------|-----------|----------------------------------------------------------|---|
| Code                    | int64     | レースの識別番号                                         |0|
| Rank                    | int64     | レースにおける順位                                       |0|
| Frame Rank              | int64     | フレーム内での順位                                       |0|
| Horse Number            | int64     | 馬の番号                                                 |0|
| Horse Name              | object    | 馬の名前                                                 |0|
| Kinryou                 | float64   | 馬の斤量（体重）                                       |0|
| Jockey                  | object    | ジョッキーの名前                                         |0|
| Time                    | float64   | レースの所要時間（秒数）                                |0|
| Chakusa                 | object    | 着順（フォーマットによる）                               |904084|
| Nobori                  | float64   | 上昇度（成績の向上を示す指標）                         |2021595|
| Tansyou                 | float64   | 単勝オッズ                                               |1659530|
| Ninki                   | float64   | 人気（馬の人気度）                                      |46|
| Horse Weight            | float64   | 馬の体重（斤量）                                       |3547|
| Trainer                 | object    | トレーナーの名前                                        |0|
| Banushi                 | object    | 馬主の名前                                              |96709|
| Shoukin                 | object    | 賞金                                                    |3167645|
| Date                    | object    | レース日（通常は文字列形式）                            |0|
| Track                   | object    | トラックの種類                                          |0|
| Weather                 | object    | 天候                                                    |0|
| Race Number             | int64     | レースの番号                                            |0|
| Distance                | int64     | レースの距離（メートル単位）                            |0|
| Condition               | object    | トラックコンディション                                   |387819|
| Disqualification        | int64     | 失格の有無（失格の場合は1、そうでなければ0）           |0|
| Sex                     | int64     | 馬の性別（数値で表現）                                  |0|
| Age                     | int64     | 馬の年齢（数値で表現）                                  |0|
| Track Type              | object    | トラックのタイプ                                        |0|
| intermediate_rank_1     | float64   | 中間順位1                                               |1072605|
| intermediate_rank_2     | float64   | 中間順位2                                               |1116477|
| intermediate_rank_3     | float64   | 中間順位3                                               |2367017|
| intermediate_rank_4     | float64   | 中間順位4                                               |3182497|
| Horse Weight Change      | int64     | 馬の体重の変化（斤量の増減）                           |0|
| Trainers Stable         | object    | トレーナーの厩舎名                                      |0|

5886227行

In [9]:
import tensorflow_probability as tfp
import tensorflow as tf

In [10]:
import pandas as pd
merged_table = pd.read_csv('./data/merged_table_finished2.csv')

In [11]:
# Chakusa, Nobori, Tansyou, Banushi, Shoukin, intermediate_rank_1, intermediate_rank_2, intermediate_rank_3, intermediate_rank_4の空カラムは削除
merged_table = merged_table.drop(['Chakusa', 'Nobori', 'Tansyou', 'Banushi', 'Shoukin', 'intermediate_rank_1', 'intermediate_rank_2', 'intermediate_rank_3', 'intermediate_rank_4'], axis=1)

# Ninki, Horse Weightカラムの欠損行を削除
merged_table = merged_table.dropna(subset=['Ninki', 'Horse Weight'])

# Conditionのnullは0に変換
merged_table['Condition'] = merged_table['Condition'].fillna(0)

In [ ]:
# 全列表示
pd.set_option('display.max_columns', None)
merged_table.head()

In [ ]:
# str型のカテゴリカルデータを数値に変換
# 1. Track
# 札幌|函館|福島|新潟|東京|中山|中京|京都|阪神|小倉|門別|盛岡|水沢|浦和|船橋|大井|川崎|金沢|笠松|名古屋|園田|姫路|高知|佐賀|帯広
# 0...24

merged_table['Track'] = merged_table['Track'].replace('札幌', 0)
merged_table['Track'] = merged_table['Track'].replace('函館', 1)
merged_table['Track'] = merged_table['Track'].replace('福島', 2)
merged_table['Track'] = merged_table['Track'].replace('新潟', 3)
merged_table['Track'] = merged_table['Track'].replace('東京', 4)
merged_table['Track'] = merged_table['Track'].replace('中山', 5)
merged_table['Track'] = merged_table['Track'].replace('中京', 6)
merged_table['Track'] = merged_table['Track'].replace('京都', 7)
merged_table['Track'] = merged_table['Track'].replace('阪神', 8)
merged_table['Track'] = merged_table['Track'].replace('小倉', 9)
merged_table['Track'] = merged_table['Track'].replace('門別', 10)
merged_table['Track'] = merged_table['Track'].replace('盛岡', 11)
merged_table['Track'] = merged_table['Track'].replace('水沢', 12)
merged_table['Track'] = merged_table['Track'].replace('浦和', 13)
merged_table['Track'] = merged_table['Track'].replace('船橋', 14)
merged_table['Track'] = merged_table['Track'].replace('大井', 15)
merged_table['Track'] = merged_table['Track'].replace('川崎', 16)
merged_table['Track'] = merged_table['Track'].replace('金沢', 17)
merged_table['Track'] = merged_table['Track'].replace('笠松', 18)
merged_table['Track'] = merged_table['Track'].replace('名古屋', 19)
merged_table['Track'] = merged_table['Track'].replace('園田', 20)
merged_table['Track'] = merged_table['Track'].replace('姫路', 21)
merged_table['Track'] = merged_table['Track'].replace('高知', 22)
merged_table['Track'] = merged_table['Track'].replace('佐賀', 23)
merged_table['Track'] = merged_table['Track'].replace('帯広', 24)

In [14]:
# 2. merged_table['Trainers Stable']は削除
merged_table = merged_table.drop(['Trainers Stable'], axis=1)

In [ ]:
# 3. Track Type : ダート|芝|障害 -> 0|1|2
merged_table['Track Type'] = merged_table['Track Type'].replace('ダート', 0)
merged_table['Track Type'] = merged_table['Track Type'].replace('芝', 1)
merged_table['Track Type'] = merged_table['Track Type'].replace('障害', 2)

In [ ]:
# 4. Weather ['晴', '曇', '雨', '小雨', '雪', '小雪'] -> 0|1|2|3|4|5
merged_table['Weather'] = merged_table['Weather'].replace('晴', 0)
merged_table['Weather'] = merged_table['Weather'].replace('曇', 1)
merged_table['Weather'] = merged_table['Weather'].replace('雨', 2)
merged_table['Weather'] = merged_table['Weather'].replace('小雨', 3)
merged_table['Weather'] = merged_table['Weather'].replace('雪', 4)
merged_table['Weather'] = merged_table['Weather'].replace('小雪', 5)

In [ ]:
# 5. Condition ['良','稍','重', '不', 0] -> 0|1|2|3|4
merged_table['Condition'] = merged_table['Condition'].replace('良', 0)
merged_table['Condition'] = merged_table['Condition'].replace('稍', 1)
merged_table['Condition'] = merged_table['Condition'].replace('重', 2)
merged_table['Condition'] = merged_table['Condition'].replace('不', 3)
merged_table['Condition'] = merged_table['Condition'].replace(0, 4)

In [ ]:
temp = merged_table.head(1)

json = temp.to_json(orient='records', lines=True)
print(json)

In [19]:
merged_table.to_csv('./data/merged_table_finished3.csv', index=False)

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

2024-11-06 00:46:01.807597: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 00:46:01.808098: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-06 00:46:01.810721: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-06 00:46:01.816239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730821561.824589  121539 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730821561.82

In [2]:
#  データの読み込み
merged_table = pd.read_csv('./data/merged_table_finished3.csv')

# merged_table['Date']カラムは削除
merged_table = merged_table.drop(['Date'], axis=1)

# Codeカラムは削除
merged_table = merged_table.drop(['Code'], axis=1)

In [ ]:
# 正規化


In [3]:
# 1. カテゴリ変数のエンコード
le_jockey = LabelEncoder()
le_trainer = LabelEncoder()
le_horsename = LabelEncoder()
merged_table["Jockey"] = le_jockey.fit_transform(merged_table["Jockey"])
merged_table["Trainer"] = le_trainer.fit_transform(merged_table["Trainer"])
merged_table["Horse Name"] = le_horsename.fit_transform(merged_table["Horse Name"])

In [4]:
# 2. 説明変数と目的変数に分ける
X = merged_table.drop(columns=["Time"])
y = merged_table["Time"]

In [5]:
# 3. 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train

In [12]:
def create_nn_model_with_embedding(numeric_input_shape):
    # JockeyのEmbedding
    jockey_input = tf.keras.layers.Input(shape=(1,), name="Jockey_Input")
    jockey_embed = tf.keras.layers.Embedding(input_dim=len(le_jockey.classes_), output_dim=4)(jockey_input)
    jockey_embed = tf.keras.layers.Flatten()(jockey_embed)

    # TrainerのEmbedding
    trainer_input = tf.keras.layers.Input(shape=(1,), name="Trainer_Input")
    trainer_embed = tf.keras.layers.Embedding(input_dim=len(le_trainer.classes_), output_dim=4)(trainer_input)
    trainer_embed = tf.keras.layers.Flatten()(trainer_embed)

    # Horse NameのEmbedding
    horsename_input = tf.keras.layers.Input(shape=(1,), name="HorseName_Input")
    horsename_embed = tf.keras.layers.Embedding(input_dim=len(le_horsename.classes_), output_dim=4)(horsename_input)
    horsename_embed = tf.keras.layers.Flatten()(horsename_embed)

    # 数値データの入力
    numeric_input = tf.keras.layers.Input(shape=(numeric_input_shape,), name="Numeric_Input")

    # 入力の結合
    x = tf.keras.layers.Concatenate()([jockey_embed, trainer_embed, horsename_embed, numeric_input])

    # 通常の Dense 層を使用
    x = tf.keras.layers.Dense(64, activation="relu")(x) # 変更点
    x = tf.keras.layers.Dense(32, activation="relu")(x) # 変更点
    output = tf.keras.layers.Dense(1)(x) # 変更点

    model = tf.keras.Model(inputs=[jockey_input, trainer_input, horsename_input, numeric_input], outputs=output)
    return model
 
# モデルのインスタンス作成
numeric_input_shape = X_train.drop(columns=["Jockey", "Trainer", "Horse Name"]).shape[1]
nn_model = create_nn_model_with_embedding(numeric_input_shape)


In [13]:
# 5. モデルのコンパイル
model.compile(
    optimizer='adam',  # オプティマイザを指定 (例: adam)
    loss='mse',  # 損失関数を指定 (例: 平均二乗誤差)
    metrics=['mae']  # 評価指標を指定 (例: 平均絶対誤差)
)

NameError: name 'model' is not defined

In [33]:
# 6. データの形式をモデルに合わせる
X_train_jockey = X_train["Jockey"].values
X_train_trainer = X_train["Trainer"].values
X_train_numeric = X_train.drop(columns=["Jockey", "Trainer"]).values
X_train_numeric = X_train_numeric.astype("float32")

X_test_jockey = X_test["Jockey"].values
X_test_trainer = X_test["Trainer"].values
X_test_numeric = X_test.drop(columns=["Jockey", "Trainer"]).values
X_test_numeric = X_test_numeric.astype("float32")

In [ ]:
# X_train_numericの要素の型を確認
print(X_train_numeric.dtype)

In [ ]:
# モデルの保存先を指定
model_filepath = "./model/best_model.h5"  # 適宜変更

# チェックポイントコールバック。`val_loss` が最小となるモデルを保存
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_filepath,
    monitor="val_loss",  # 検証データの損失を監視
    save_best_only=True,  # 最良のモデルのみを保存
    save_weights_only=False, # モデル全体を保存
    verbose=1  # 保存時にメッセージを表示
)

# 早期終了コールバック。`val_loss` が改善しなくなったら学習を停止
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",  # 検証データの損失を監視
    patience=5,  # 改善が見られないエポック数を指定 (例: 5エポック)
    verbose=1
)

# 7. モデルの学習
history = model.fit( # bnn_model を model に変更
    [X_train_jockey, X_train_trainer, X_train_numeric],
    y_train,
    epochs=50, # エポック数は必要に応じて調整
    batch_size=16, # バッチサイズは必要に応じて調整
    validation_data=([X_test_jockey, X_test_trainer, X_test_numeric], y_test)
    callbacks=[checkpoint_callback, early_stopping_callback]  # コールバックを指定
)

In [ ]:

# 8. 評価
loss, mae = bnn_model.evaluate([X_test_jockey, X_test_trainer, X_test_numeric], y_test)
print(f"Mean Absolute Error: {mae}")